# Simple CNN for MNIST Digit Recognizing

It's just my first notebook on Kaggle... creating a CNN for classifying the MNIST digit number set (like it's probably done by many other kagglers)

## Load train data.

In [ ]:
import keras
import pandas as pd

# Load the data
df = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")

## Prepare data.
First let's split the dataframe in predictors and labels. Also get dummy variables for each of the 0-9 label values. Reformat the training set in 28x28 matrices in order to use them as predictors for our CNN.

In [ ]:
import numpy as np
from keras.utils import to_categorical

y_train = df["label"]
y_train = to_categorical(y_train)

X_train = df.drop("label", axis=1)
X_train = np.array(X_train).reshape(-1, 28, 28)

print(y_train.shape)
print(X_train.shape)

Just for interest: take a look at a random input value and check the assigned label.

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

i = 6541

plt.imshow(X_train[i])
plt.show()
print(y_train[i])

The resulting image look like a "7" what is also indicated by our label array.

### Image shifting.

We create some addtional training data, through shifting our already observed data.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

gen = ImageDataGenerator(width_shift_range=3, height_shift_range=3)
X_train_reshaped = X_train.reshape(-1, 28, 28,1)
gen.fit(X_train_reshaped)

# Print one shifted digit
for batch in gen.flow(X_train_reshaped, y_train, shuffle=True):
    first_image = batch[0][0]
    plt.imshow(first_image.reshape(-1, 28, 28)[0])
    plt.show()
    break

X_train_shifted = gen.flow(X_train.reshape(-1, 28, 28,1), y_train, shuffle=True)

Now, create, compile and train our CNN.

In [ ]:
import math

from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout
from keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping

early_stopping_monitor = EarlyStopping(patience=10, monitor="loss")

model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(28, 28, 1), activation="relu", padding="same"))
model.add(Conv2D(32, kernel_size=(3, 3), activation="relu", padding="same"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=(3, 3), activation="relu", padding="same"))
model.add(Conv2D(64, kernel_size=(3, 3), activation="relu", padding="same"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation="relu", padding="same"))
model.add(Conv2D(128, kernel_size=(3, 3), activation="relu", padding="same"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dense(128, activation="relu"))

model.add(Dense(10, activation="sigmoid"))

model.compile(optimizer=RMSprop(lr=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

model.fit(
    X_train.reshape(-1, 28, 28, 1),
    y_train,
    epochs=9999,
    batch_size=1000,
    callbacks=early_stopping_monitor
)

Let's check the accuracy on our whole training set.

In [ ]:
res_train = model.evaluate(X_train.reshape(-1, 28, 28, 1), y_train)
for i in range(0, len(res_train)):
    print(model.metrics_names[i] + ": " + str(res_train[i]))

Create test dataframe and show shape.

In [ ]:
df = pd.read_csv("../input/digit-recognizer/test.csv")
df.shape

Everything looks nice... finally let's create our test data, use our CNN to predict digits and turn the result into the correct format.

In [ ]:
X_test = np.array(df).reshape(-1, 28, 28)
res = model.predict_classes(X_test.reshape(-1, 28, 28, 1))
res.shape

In [ ]:
res_df = pd.DataFrame(res, index=range(1, 28001), columns=["Label"])
res_df.index.name="ImageId"
print(res_df)
res_df.to_csv("result.csv")